In [2]:
import pandas as pd
%matplotlib inline
import requests
import json
import demjson
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
from weather import Weather

ModuleNotFoundError: No module named 'weather'

In [ ]:
Weather('温州').plot_weather('201701', '201906')

In [ ]:
date = '201905'
weather = requests.get('http://tianqi.2345.com/t/wea_history/js/{date}/59493_{date}.js'.format(date=date)).text.split('=')[1][:-1]
weather = demjson.decode(weather)['tqInfo']
pd.DataFrame(weather)
               

In [ ]:
def read_weather(start, end):
    dates = date_generate(start, end)
    return pd.concat(map(get_weather, dates))
def get_weather(date):
    weather = requests.get('http://tianqi.2345.com/t/wea_history/js/{date}/59493_{date}.js'.format(date=date)).text.split('=')[1][:-1]
    weather = demjson.decode(weather)['tqInfo']
    weather = json.dumps(weather)
    df = pd.read_json(weather, orient='records')
    df['month'] = date
    return df
               

In [ ]:
get_weather(201801)

In [ ]:
def date_generate(start, end):
    start = datetime.strptime(start, '%Y%m')
    end = datetime.strptime(end, '%Y%m')
    while True:
        next_start = start + relativedelta(months=1)
        yield start.strftime('%Y%m')
        if next_start > end:
            break
        start = next_start


In [ ]:
df = read_weather('201801', '201812')
df

In [ ]:
rain = '雨'
rain_index = ' ' + rain
cloudy = '多云'
cloudy_index = ' ' + cloudy
overcast = '阴'
overcast_index = ' ' + overcast
sunny = '晴'
sunny_index = ' ' + sunny
def weath_categoty(row):
    tianqi = row['tianqi']
    if tianqi.find(rain) != -1:
        return rain_index
    if tianqi.find(overcast) != -1:
        return overcast_index
    if tianqi.find(cloudy) != -1:
        return cloudy_index
    return sunny_index


In [ ]:
def save_weather(start, end, city):
    df = read_weather(start, end).dropna().reset_index()
    df.to_csv('./resources/weather/{}.csv'.format(city))
def plot_weather(start, end):
    df = read_weather(start, end).dropna().reset_index()
    df['weather'] = df.apply(weath_categoty, axis=1)
    
    from pylab import rcParams
    rcParams['figure.figsize'] = 40, 10
    weather_df = df.groupby(['month', 'weather']).aqi.count().unstack().reset_index()
    weather_df.plot.bar(x='month', y=[rain_index, overcast_index, cloudy_index, sunny_index])

In [ ]:
plot_weather('201701', '201906')

In [ ]:
df = df.dropna().reset_index()
df['weather'] = df.dropna().apply(weath_categoty, axis=1)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
weather_df = df.groupby(['month', 'weather']).aqi.count().unstack().reset_index()
weather_df.plot.bar(x='month', y=[rain_index, overcast_index, cloudy_index, sunny_index])